In [64]:
from embedder import Embedder
from data_provider import DataProvider

In [65]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/mounts/data/proj/faeze/.cache/hf/'
os.environ['HF_HOME'] = '/mounts/data/proj/faeze/.cache/hf/'
os.environ['HF_DATASETS_CACHE'] = '/mounts/data/proj/faeze/.cache/hf/'
os.environ['TORCH_HUB'] = '/mounts/data/proj/faeze/.cache/torch/'
os.environ['TORCH_HOME'] = '/mounts/data/proj/faeze/.cache/torch/'
os.environ["WANDB_DIR"] = '/mounts/data/proj/faeze/.cache/wandb/'

In [66]:
data_provider_ = DataProvider()

In [67]:
datas = ["bas19_es", "for19_pt", "has21_hi", "ous19_ar", "ous19_fr", "san20_it", "gahd24_de", "xdomain_tr"]
languages = [i.split('_')[-1] for i in datas]

In [68]:
datasets = data_provider_.load_datasets(datas, languages)

Using the latest cached version of the module from /mounts/data/corp/huggingface/modules/datasets_modules/datasets/baseline_data/3938a7c6396928f6c92ae06c70ab520777a86d4ff32acff80c7e6012e41492ac (last modified on Fri Jan 17 20:44:55 2025) since it couldn't be found locally at baseline_data, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /mounts/data/corp/huggingface/modules/datasets_modules/datasets/baseline_data/3938a7c6396928f6c92ae06c70ab520777a86d4ff32acff80c7e6012e41492ac (last modified on Fri Jan 17 20:44:55 2025) since it couldn't be found locally at baseline_data, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /mounts/data/corp/huggingface/modules/datasets_modules/datasets/baseline_data/3938a7c6396928f6c92ae06c70ab520777a86d4ff32acff80c7e6012e41492ac (last modified on Fri Jan 17 20:44:55 2025) since it couldn't be found locally at baseline_data, or remotely on the Hugging Face Hub.
Using the latest

In [69]:
datasets

[{'name': 'bas19_es',
  'data': DatasetDict({
      train: Dataset({
          features: ['id', 'text', 'label'],
          num_rows: 4100
      })
      validation: Dataset({
          features: ['id', 'text', 'label'],
          num_rows: 500
      })
      test: Dataset({
          features: ['id', 'text', 'label'],
          num_rows: 2000
      })
      hate_check: Dataset({
          features: ['id', 'text', 'label'],
          num_rows: 3745
      })
  }),
  'language': 'es'},
 {'name': 'for19_pt',
  'data': DatasetDict({
      train: Dataset({
          features: ['id', 'text', 'label'],
          num_rows: 3170
      })
      validation: Dataset({
          features: ['id', 'text', 'label'],
          num_rows: 500
      })
      test: Dataset({
          features: ['id', 'text', 'label'],
          num_rows: 2000
      })
      hate_check: Dataset({
          features: ['id', 'text', 'label'],
          num_rows: 3691
      })
  }),
  'language': 'pt'},
 {'name': 'has21_hi',


In [70]:

embedder_ = Embedder(model_name="m3")
embeddings, metadatas = embedder_.embed_datasets(datasets, ['test'], False)

In [71]:
import numpy as np
embed = np.vstack(embeddings)
metadata = metadatas

In [72]:
len(metadata), embed.shape

(14500, (14500, 1024))

In [73]:
embed.shape

(14500, 1024)

In [74]:
# clusters = Embedder.cluster_embeddings(embed, metadata, 150)
# clusters

In [75]:
embed.shape, len(metadata)

((14500, 1024), 14500)

In [76]:
Embedder.visualize_embeddings(embed, metadatas, reduction_method="tsne")

In [77]:
import pandas as pd
import numpy as np
metadata_df = pd.DataFrame(metadatas)

In [78]:
metadata_df['dataset_name'].value_counts()

dataset_name
bas19_es      2000
for19_pt      2000
has21_hi      2000
san20_it      2000
gahd24_de     2000
xdomain_tr    2000
ous19_fr      1500
ous19_ar      1000
Name: count, dtype: int64

In [79]:
from retriever import Retriever
retriever = Retriever(embedder_.embedding_dim, index_type="HNSW")

In [80]:
retriever.device

'cuda'

In [81]:
retriever.load_index("/mounts/data/proj/faeze/data_efficient_hate/models/retriever/all_multilingual_with_m3/")

In [82]:
# retrieved_samples = retriever.retrieve()

In [83]:
# retriever.save_index("/mounts/data/proj/faeze/data_efficient_hate/models/retriever/temp/")

In [84]:
from collections import Counter

for i, data in enumerate(datas):
    embedd = embeddings[i]
    retrieved = retriever.retrieve_multiple_queries(embedd, num_retrieved=500, exclude_datasets=[data])

    # Extract relevant metadata
    dataset_names = [item['metadata']['dataset_name'] for item in retrieved]
    languages = [item['metadata']['language'] for item in retrieved]
    labels = [item['metadata']['label'] for item in retrieved]

    # Count occurrences
    dataset_counter = Counter(dataset_names)
    language_counter = Counter(languages)
    label_counter = Counter(labels)

    # print(data, dataset_counter)

    # Find most common and calculate percentages
    most_common_dataset, dataset_count = dataset_counter.most_common(1)[0]
    most_common_language, language_count = language_counter.most_common(1)[0]
    most_common_label, label_count = label_counter.most_common(1)[0]

    total_retrieved = len(retrieved)
    dataset_percentage = (dataset_count / total_retrieved) * 100
    language_percentage = (language_count / total_retrieved) * 100
    label_percentage = (label_count / total_retrieved) * 100

    # Print results
    print(data)
    print(f"Most retrieved dataset_name: {most_common_dataset} ({dataset_percentage:.2f}%)")
    print(f"Most retrieved language: {most_common_language} ({language_percentage:.2f}%)")
    print(f"Most retrieved label: {most_common_label} ({label_percentage:.2f}%)")
    print()


bas19_es
Most retrieved dataset_name: fou18_en (19.00%)
Most retrieved language: en (55.80%)
Most retrieved label: 0 (56.20%)

for19_pt
Most retrieved dataset_name: fou18_en (31.80%)
Most retrieved language: en (59.40%)
Most retrieved label: 0 (72.00%)

has21_hi
Most retrieved dataset_name: fou18_en (20.60%)
Most retrieved language: en (43.80%)
Most retrieved label: 0 (60.80%)

ous19_ar
Most retrieved dataset_name: xdomain_tr (17.80%)
Most retrieved language: en (49.80%)
Most retrieved label: 0 (62.00%)

ous19_fr
Most retrieved dataset_name: fou18_en (23.60%)
Most retrieved language: en (53.20%)
Most retrieved label: 0 (64.60%)

san20_it
Most retrieved dataset_name: ken20_en (18.20%)
Most retrieved language: en (58.40%)
Most retrieved label: 1 (50.00%)

gahd24_de
Most retrieved dataset_name: dyn21_en (99.40%)
Most retrieved language: en (100.00%)
Most retrieved label: 1 (73.80%)

xdomain_tr
Most retrieved dataset_name: xdomain_en (35.00%)
Most retrieved language: en (78.80%)
Most retri

In [86]:
from collections import defaultdict

for i, data in enumerate(datas):
    embedd = embeddings[i]
    print(data)
    retrieved = retriever.retrieve_multiple_queries(embedd, num_retrieved=500, exclude_datasets=[data])

    scores = [item['score'] for item in retrieved]

    # Calculate average distances for datasets, languages, and labels
    dataset_scores = defaultdict(list)
    language_scores = defaultdict(list)
    label_scores = defaultdict(list)

    for item, score in zip(retrieved, scores):
        dataset_scores[item['metadata']['dataset_name']].append(score)
        language_scores[item['metadata']['language']].append(score)
        label_scores[item['metadata']['label']].append(score)

    # Compute average distances
    avg_dataset_scores = {k: sum(v) / len(v) for k, v in dataset_scores.items()}
    avg_language_scores = {k: sum(v) / len(v) for k, v in language_scores.items()}
    avg_label_scores = {k: sum(v) / len(v) for k, v in label_scores.items()}

    # Find closest group based on minimum average distance
    closest_dataset = min(avg_dataset_scores, key=avg_dataset_scores.get)
    closest_language = min(avg_language_scores, key=avg_language_scores.get)
    closest_label = min(avg_label_scores, key=avg_label_scores.get)

    # Print results
    print(f"Closest dataset_name (by score): {closest_dataset} (avg distance: {avg_dataset_scores[closest_dataset]:.4f})")
    print(f"Closest language (by score): {closest_language} (avg distance: {avg_language_scores[closest_language]:.4f})")
    print(f"Closest label (by score): {closest_label} (avg distance: {avg_label_scores[closest_label]:.4f})")


bas19_es
Closest dataset_name (by score): has21_hi (avg distance: 0.4574)
Closest language (by score): hi (avg distance: 0.4574)
Closest label (by score): 0 (avg distance: 0.5079)
for19_pt
Closest dataset_name (by score): bas19_es (avg distance: 0.4466)
Closest language (by score): es (avg distance: 0.4466)
Closest label (by score): 0 (avg distance: 0.4637)
has21_hi
Closest dataset_name (by score): ous19_fr (avg distance: 0.5193)
Closest language (by score): fr (avg distance: 0.5193)
Closest label (by score): 0 (avg distance: 0.5801)
ous19_ar
Closest dataset_name (by score): for19_pt (avg distance: 0.5605)
Closest language (by score): pt (avg distance: 0.5605)
Closest label (by score): 0 (avg distance: 0.6369)
ous19_fr
Closest dataset_name (by score): has21_hi (avg distance: 0.4705)
Closest language (by score): hi (avg distance: 0.4705)
Closest label (by score): 0 (avg distance: 0.4944)
san20_it
Closest dataset_name (by score): ous19_fr (avg distance: 0.5304)
Closest language (by score